# Administrivia

## Timetable

* Week 4 - Week 4 lab released, week 3 lab due.
* Week 5 - Week 5 lab released, week 4 lab due.
* Week 6 - No lab released, no lab due
* Week 7 - No lab released, project proposal due (Sunday at end of week)
* Week 8 - Week 8 lab released (short), week 5 lab due.
* Week 9 - Week 9 lab released (short), week 8 lab due.
* Week 10 - No lab released, week 9 lab due, project final prototype due.

## Week 6 Revision session
* Who can attend in person?
* Tuesday or Thursday lecture time?

## Project
* You're not expected to know how to build web apps yet!
* Questions?

# What am I looking at?

This is Visual Studio Code.

We'll see how it works as we go.

In [ ]:
1 + 1

# Lab & Dictionary Recap

This list contains the email addresses of 4 different students

In [ ]:
emails = ["z1234567@student.unsw.edu.au", "z7654321@unsw.edu.au", "z7891234@ad.unsw.edu.au", "z1357924@student.unsw.edu.au"]

Can we build a dictionary containing the ZIDs as keys and email addresses as values?

In [ ]:
zid_to_email = {}

for email in emails:
    zid = email[0:8]
    zid_to_email[zid] = email

zid_to_email

How many students prefer each email domain?

In [ ]:
domain_popularity = {}

for email in emails:
    domain = email[9:]
    if domain in domain_popularity:
        domain_popularity[domain] = domain_popularity[domain] + 1
    else:
        domain_popularity[domain] = 1

domain_popularity

Can we group students by their preferred email domain?

In [ ]:
domain_to_students = {}

for email in emails:
    domain = email[9:]
    zid = email[:8]
    if domain in domain_to_students:
        domain_to_students[domain].append(zid)
    else:
        domain_to_students[domain] = [zid]

domain_to_students

# Musk tweets

This code cell creates a list from some of Elon Musk's tweets. 

*You're not expected to completely understand this code yet. It's only here to give us interesting data*.

In [ ]:
import pandas

raw_tweets = pandas.read_csv('https://raw.githubusercontent.com/robeverest/cs1010/master/data/elonmusk-full.csv')
elon_tweets = list(map(str, list(raw_tweets.tweet)))

## Tweet anaylsis

In [ ]:
example_tweet = "the cat sat on the mat. the cat slept"

# the cat -> sat, cat sat -> on, sat on -> the, on the -> mat., the mat. -> the, mat. the -> cat, the cat -> slept

example_result = { None: { "the": 1 },
                  "the": { "cat": 1},
                  "the cat": { "sat": 1, "slept": 1 }, 
                  "cat sat": { "on": 1 }, 
                  "sat on": { "the": 1 },
                  "on the": { "mat.": 1 },
                  "the mat.": { "the": 1 },
                  "mat. the": { "cat": 1 },
                  "the cat": { "slept": 1 } }

In [ ]:
elon_word_map = {}
elon_word_map[tuple()] = {}

for tweet in elon_tweets:
    prefix = tuple()
    for word in tweet.split():
        # Have we seen this prefix before?
        if not prefix in elon_word_map:
            # If we haven't, add an entry to the word map
            elon_word_map[prefix] = {}
        # Have we seen this word follow the prefix?
        if word in elon_word_map[prefix]:
            # If it has, increase the count by 1
            elon_word_map[prefix][word] += 1
        else:
            # If it hasn't, set the count to 1
            elon_word_map[prefix][word] = 1

        if len(prefix) == 0:
            # If we don't have a prefix yet, the prefix is just one word
            prefix = (word,)
        elif len(prefix) == 1:
            # If our prefix is already one word, we make it the two words
            prefix = (prefix[0], word)
        else:
            # If our prefix is already 2 words, remove the first word and add the new word to the end
            prefix = (prefix[1], word)

elon_word_map

Generating artificial tweets?

In [ ]:
import random

def random_word(words_weight):
    if words_weight == {}:
        return None
    words = random.choices(list(words_weight.keys()), weights=list(words_weight.values()))
    return words[0]

def generate_tweet():
    word = random_word(elon_word_map[tuple()])
    tweet = ""
    prefix = tuple()
    while word != None and len(tweet + word) < 280:
        tweet = tweet + word + " "
        if len(prefix) == 0:
            # If we don't have a prefix yet, the prefix is just one word
            prefix = (word,)
        elif len(prefix) == 1:
            # If our prefix is already one word, we make it the two words
            prefix = (prefix[0], word)
        else:
            # If our prefix is already 2 words, remove the first word and add the new word to the end
            prefix = (prefix[1], word)

        if prefix in elon_word_map:
            # There is a word we can use
            word = random_word(elon_word_map[prefix])
        else:
            # No words follow on from this prefix
            word = None
    if word != None:
        tweet = tweet.strip() + '...'
    return tweet.strip()

generate_tweet()

What happens if we try to generate tweets using a prefix length of 3 or higher?

In [ ]:
def generate_word_map(prefix_length):
    elon_word_map = {}
    for tweet in elon_tweets:
        prefix = tuple()
        for word in tweet.split():
            # Have we seen this prefix before?
            if not prefix in elon_word_map:
                # If we haven't, add an entry to the word map
                elon_word_map[prefix] = {}
            # Have we seen this word follow the prefix?
            if word in elon_word_map[prefix]:
                # If it has, increase the count by 1
                elon_word_map[prefix][word] += 1
            else:
                # If it hasn't, set the count to 1
                elon_word_map[prefix][word] = 1

            # Add word to prefix
            prefix = prefix + (word,)
            if len(prefix) > prefix_length:
                # Remove starting word from prefix
                prefix = prefix[1:]
    return elon_word_map

generate_word_map(3)

In [16]:
import random

def random_word(words_weight):
    if words_weight == {}:
        return None
    words = random.choices(list(words_weight.keys()), weights=list(words_weight.values()))
    return words[0]

def generate_tweet(word_map, prefix_length):
    word = random_word(word_map[tuple()])
    tweet = ""
    prefix = tuple()
    while word != None and len(tweet + word) < 280:
        tweet = tweet + word + " "
        prefix = prefix + (word,)
        if len(prefix) > prefix_length:
            prefix = prefix[1:]

        if prefix in word_map:
            # There is a word we can use
            word = random_word(word_map[prefix])
        else:
            # No words follow on from this prefix
            word = None
    if word != None:
        tweet = tweet.strip() + '...'
    return tweet.strip()

elon_word_map_3 = generate_word_map(3)
fake_tweet = generate_tweet(elon_word_map_3, 3)
print(fake_tweet)
fake_tweet in elon_tweets

Makes no sense. In order to grow the colony, you’d have to transport vast amounts of video data. It's a beast! Please consider joining our AI or computer/chip teams if this sounds interesting.


False

In [28]:
real_tweets = 0

prefix_length = 2
word_map = generate_word_map(prefix_length)
for _ in range(1000):
    tweet = generate_tweet(word_map, prefix_length)
    if tweet in elon_tweets:
        real_tweets += 1

real_tweets/1000

0.253

# Algorithms

An algorithm is a sequence of well-defined steps to solve particular problem.

In this course we have already *implemented* many algorithms.

In [29]:
haystack = ['needle', 'hay', 'needle', 'hay', 'hay']

needles = 0
for long_thing in haystack:
    if long_thing == 'needle':
        needles += 1

needles

2

## Greatest Common Divisor

The greatest common divisor of numbers $a$ and $b$ is the largest number that divides both of them without remainder.

This is a problem for which we can come up with multiple different algorithms (and implementations).



In [ ]:
gcd(12, 15) = 3

### Modulo operator

The modulo operator (`%`) in Python lets us compute what the remainder of division operations.

What is the remainder of dividing 25 by 7?

In [30]:
25 % 7

4

Is 8298973 divisible by 2591

In [31]:
8298973 % 2591

0

What's the simplest way to find the GCD of two numbers?

In [33]:
def gcd(a, b):
    result = 1
    for i in range(1,a):
        if a % i == 0 and b % i == 0:
            result = i
    return result

gcd(8298973, 2591)

2591

What's the GCD of 157680487 and 190876379?

In [34]:
gcd(157680487, 190876379)

8298973

### Euclid's algorithm

Euclid's algorithm is a fast way of calculating the GCD of two numbers.

Implement Euclid's algorithm using subtraction

In [35]:
def gcd(a, b):
    while a != b:
        if a > b:
            a = a - b
        else:
            b = b - a
    return a
gcd(157680487, 190876379)

8298973

Implement Euclid's algorithm using the modulo operator

In [36]:
def gcd(a, b):
    while b > 0:
        t = b
        b = a % b
        a = t
    return a
gcd(157680487, 190876379)

8298973